## OpenAltimetry IceSat 2 Workflows

### Overview

This notebook uses [ATL03](https://nsidc.org/data/ATL03/versions/1) data from the IceSat 2 mission via the OpenAltimetry API for subsetting. This API enable users to select a particular region of interest and get pre-processed photon data wihtout the need to deal with big HDF files.

### Instructions

The only thing you need for this notebook to work is a parameter's list that's available on the OpenAltimetry photon chart, paste that string as the OA_REFERENCE_URL and start visualizing your data!

Note that this notebook already has the Python libraries that you might need to do data analysis such as geopandas, plotly, fiona, shapely and more!

In [ ]:
import numpy as np
import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import requests
import pyproj
import math
import plotly.graph_objects as go
from plotly.offline import iplot


# Paste the OpenAltimetry selection parameters here
OA_REFERENCE_URL = 'minx=72.0476&miny=36.6165&maxx=72.0505&maxy=36.6214&date=2019-05-02&trackId=516'
# We populate a list with the photon data using the OpenAltimetry API, no HDF! 
OA_URL = 'https://openaltimetry.org/data/icesat2/getPhotonData?client=jupyter&' + OA_REFERENCE_URL

OA_PHOTONS = ['Noise', 'Low', 'Medium', 'High']

df_beams = []
oa_plots = []

# This function will request the 6 tracks using OpenAltimetry's API
def getPhotonData():
    series = []
    beam_index = 0;
    for beam in range(6):
        beam_index += 1
        b_url = OA_URL + '&beam=' + str(beam + 1)
        print(b_url)
        print('Requesting Beam {}'.format(beam_index))
        r = requests.get(b_url)
        data = r.json()
        series.append(data)
    print ('Data requests completed you can now plot it on the next cell...')
    return series

        
photon_cloud = getPhotonData()

track_index = 0
for track in photon_cloud:
    track_index +=1
    for photons in track['series']:
        if any(word in photons['name'] for word in OA_PHOTONS):
            series = []
            for p in photons['data']:
                series.append({
                    'lat': p[0],
                    'lon': p[2],
                    'h': p[1]
                })
            if (len(series) > 0):
                df = pd.DataFrame.from_dict(series)
                df.name = 'Track ' + str(track_index) + ' ' + photons['name']
                df_beams.append(df) 

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'], mode='markers',
                                    marker=dict(
                                        size=1,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    )
                                )
                   )
    



In [ ]:
layout = go.Layout(
    width=900,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

fig = go.Figure(data=oa_plots, layout=layout)  

iplot(fig)

In [ ]:
# Now we can get a histogram of the beam's elevation and their description
for df in df_beams:
    print(df.describe())
    df.hist('h', bins=128)
    